<a href="https://colab.research.google.com/github/hamzzihyun/competition/blob/main/DACON/%5BLG%20Aimers%5D%EC%8A%A4%EB%A7%88%ED%8A%B8%EA%B3%B5%EC%9E%A5%20_%EC%A0%9C%ED%92%88_%ED%92%88%EC%A7%88_%EC%83%81%ED%83%9C_%EB%B6%84%EB%A5%98_AI%EC%98%A8%EB%9D%BC%EC%9D%B8%ED%95%B4%EC%BB%A4%ED%86%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/DACON/제품품질분류ai/open.zip') as z:
  z.extractall()

## Import

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

## Data Load

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = train[['Y_Class','Y_Quality','LINE','PRODUCT_CODE']]

In [ ]:
# Y_Class
df['Y_Class'].value_counts()

1    407
2    103
0     88
Name: Y_Class, dtype: int64

In [ ]:
# 제품 품질 상태당 Y_Quality 통계량
print('적정 기준 미달(부적합)')
print(df['Y_Quality'][df['Y_Class'] == 0].describe().T)
print('\n적합')
print(df['Y_Quality'][df['Y_Class'] == 1].describe().T)
print('\n적정 기준 초과(부적합)')
print(df['Y_Quality'][df['Y_Class'] == 2].describe().T)

적정 기준 미달(부적합)
count    88.000000
mean      0.520837
std       0.004603
min       0.500856
25%       0.519440
50%       0.522054
75%       0.524113
max       0.525067
Name: Y_Quality, dtype: float64

적합
count    407.000000
mean       0.530253
std        0.002444
min        0.525086
25%        0.528378
50%        0.530308
75%        0.532129
max        0.534843
Name: Y_Quality, dtype: float64

적정 기준 초과(부적합)
count    103.000000
mean       0.542031
std        0.008038
min        0.534951
25%        0.536354
50%        0.538508
75%        0.546555
max        0.578841
Name: Y_Quality, dtype: float64


In [ ]:
# LINE
df['LINE'].value_counts()

T100304    175
T100306    174
T050304     78
T010306     70
T010305     59
T050307     42
Name: LINE, dtype: int64

In [ ]:
print(df[['Y_Class','LINE']][df['LINE'] == 'T100304'].value_counts())
print(df[['Y_Class','LINE']][df['LINE'] == 'T100306'].value_counts())
print(df[['Y_Class','LINE']][df['LINE'] == 'T050304'].value_counts())
print(df[['Y_Class','LINE']][df['LINE'] == 'T010306'].value_counts())
print(df[['Y_Class','LINE']][df['LINE'] == 'T010305'].value_counts())
print(df[['Y_Class','LINE']][df['LINE'] == 'T050307'].value_counts())

Y_Class  LINE   
1        T100304    144
0        T100304     17
2        T100304     14
dtype: int64
Y_Class  LINE   
1        T100306    145
2        T100306     18
0        T100306     11
dtype: int64
Y_Class  LINE   
1        T050304    37
0        T050304    30
2        T050304    11
dtype: int64
Y_Class  LINE   
1        T010306    37
2        T010306    27
0        T010306     6
dtype: int64
Y_Class  LINE   
1        T010305    35
0        T010305    13
2        T010305    11
dtype: int64
Y_Class  LINE   
2        T050307    22
0        T050307    11
1        T050307     9
dtype: int64


- T100~ : 상대적으로 적합(1) 제품이 많음 (0 28개, 1 289개, 2 32개)
- T050~ : (0 41개, 1 46개, 2 33개)
- T010~ : (0 19개, 1 72개, 2 38개)

In [ ]:
# PRODUCT_CODE
df['PRODUCT_CODE'].value_counts()

T_31    343
A_31    249
O_31      6
Name: PRODUCT_CODE, dtype: int64

In [ ]:
print(df[['Y_Class','LINE','PRODUCT_CODE']][df['PRODUCT_CODE'] == 'T_31'].value_counts())
print(df[['Y_Class','LINE','PRODUCT_CODE']][df['PRODUCT_CODE'] == 'A_31'].value_counts())
print(df[['Y_Class','LINE','PRODUCT_CODE']][df['PRODUCT_CODE'] == 'O_31'].value_counts())

Y_Class  LINE     PRODUCT_CODE
1        T100306  T_31            144
         T100304  T_31            141
0        T100304  T_31             17
2        T100306  T_31             16
         T100304  T_31             14
0        T100306  T_31             11
dtype: int64
Y_Class  LINE     PRODUCT_CODE
1        T010306  A_31            37
         T050304  A_31            37
         T010305  A_31            35
0        T050304  A_31            30
2        T010306  A_31            27
         T050307  A_31            22
0        T010305  A_31            13
         T050307  A_31            11
2        T010305  A_31            11
         T050304  A_31            11
1        T050307  A_31             9
0        T010306  A_31             6
dtype: int64
Y_Class  LINE     PRODUCT_CODE
1        T100304  O_31            3
2        T100306  O_31            2
1        T100306  O_31            1
dtype: int64


In [ ]:
# T_31은 (0 28개, 1 285개, 2 30개)
# A_31은 (0 60개, 1 118개, 2 71개)
# O_31은 (0 0개, 1 4개, 2 2개)
# 적합이 많은 코드 T_31
# 미달이 많은 코드 A_31
# 초과가 많은 코드 A_31

- T_31 , O_31 코드는 전부 T100~ 라인 (적합 많음)
- A_31 코드는 T010, T050 라인

In [ ]:
# T100 라인의 Y_Quality 확인
# T010 라인의 Y_Quality 확인
# T050 라인의 Y_Queality 확인

**적합 제품이 많은 T100 생산라인**

In [ ]:
df[['Y_Quality']][df['LINE'] == 'T100304'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,175.0,0.529918,0.004701,0.507571,0.528039,0.530187,0.532382,0.545678


In [ ]:
df[['Y_Quality']][df['LINE'] == 'T100306'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,174.0,0.530735,0.004748,0.502517,0.528411,0.530533,0.53253,0.551279


**T010 생산라인**

In [ ]:
df[['Y_Quality']][df['LINE'] == 'T010306'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,70.0,0.534307,0.009074,0.513135,0.528813,0.533176,0.538079,0.560317


In [ ]:
df[['Y_Quality']][df['LINE'] == 'T010305'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,59.0,0.530675,0.00878,0.500856,0.526698,0.530497,0.534221,0.55877


**T050 생산라인**

In [ ]:
df[['Y_Quality']][df['LINE'] == 'T050304'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,78.0,0.527951,0.008371,0.513751,0.521655,0.527643,0.531569,0.560568


In [ ]:
df[['Y_Quality']][df['LINE'] == 'T050307'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,42.0,0.535738,0.012841,0.515884,0.524857,0.537443,0.541365,0.578841


LINE 별로 모델을 만들면 어떨지?

PRODUCT_CODE 별로 모델을 만들면 어떨지?

Y_Quality 를 라인별, 코드별과 합쳐서 column을 생성할지?

In [ ]:
df[['Y_Quality']][df['PRODUCT_CODE'] == 'T_31'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,343.0,0.530297,0.004753,0.502517,0.528153,0.530308,0.532405,0.551279


In [ ]:
df[['Y_Quality']][df['PRODUCT_CODE'] == 'A_31'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,249.0,0.531697,0.009965,0.500856,0.525221,0.530606,0.536206,0.578841


In [ ]:
df[['Y_Quality']][df['PRODUCT_CODE'] == 'O_31'].describe().T

,count,mean,std,min,25%,50%,75%,max
Y_Quality,6.0,0.531947,0.003505,0.525916,0.530744,0.532538,0.534638,0.535205


- Y_Class별 0.52는 미달 0.53은 적정, 0.54는 초과 느낌?
- 코드별로 보면 가장 작은 퀄리티는 0.500, 가장 높은건 0.578

## Include TIMESTAMP variable

In [ ]:
df2 = train[['Y_Class','Y_Quality','TIMESTAMP','LINE','PRODUCT_CODE']]

In [ ]:
print(df2['TIMESTAMP'].head(5))
print()
print(df2['TIMESTAMP'].tail(5))

0    2022-06-13 5:14
1    2022-06-13 5:22
2    2022-06-13 5:30
3    2022-06-13 5:39
4    2022-06-13 5:47
Name: TIMESTAMP, dtype: object

593    2022-09-08 14:30
594    2022-09-08 22:38
595    2022-09-08 22:47
596    2022-09-08 14:38
597    2022-09-08 14:46
Name: TIMESTAMP, dtype: object


In [ ]:
# TIMESTAMP 데이터타입 변환
# train['TIMESTAMP'] = pd.to_datetime(train['TIMESTAMP']).dt.strftime('%m').astype('int64')
# test['TIMESTAMP'] = pd.to_datetime(test['TIMESTAMP']).dt.strftime('%m').astype('int64')

In [ ]:
# Y_Class별 TIMESTAMP 확인

In [ ]:
# Y_Quality별 TIMESTAMP 확인

In [ ]:
# 생산라인별 TIMESTAMP 확인

In [ ]:
# 코드별 TIMESTAMP 확인

## train_x Data Load

In [ ]:
train_x = train.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
print(train_x.shape)
print(train_x.columns)
train_x.head()

(598, 2877)
Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7',
       'X_8',
       ...
       'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
       'X_2873', 'X_2874', 'X_2875'],
      dtype='object', length=2877)


,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check Na
train_x.isna().sum()

LINE              0
PRODUCT_CODE      0
X_1             249
X_2             249
X_3             249
               ... 
X_2871          499
X_2872          598
X_2873          598
X_2874          598
X_2875          598
Length: 2877, dtype: int64

In [ ]:
train_x.dtypes

LINE             object
PRODUCT_CODE     object
X_1             float64
X_2             float64
X_3             float64
                 ...   
X_2871          float64
X_2872          float64
X_2873          float64
X_2874          float64
X_2875          float64
Length: 2877, dtype: object

In [ ]:
# 비식별 X변수
train_x.iloc[:, 2:].describe().T

,count,mean,std,min,25%,50%,75%,max
X_1,349.0,2.409742,5.895256,1.0,2.0,2.0,2.0,103.0
X_2,349.0,95.123209,4.107640,87.0,93.0,95.0,98.0,102.0
X_3,349.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
X_4,349.0,45.000000,0.000000,45.0,45.0,45.0,45.0,45.0
X_5,349.0,10.392550,0.489019,10.0,10.0,10.0,11.0,11.0
...,...,...,...,...,...,...,...,...
X_2871,99.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
X_2872,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X_2873,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X_2874,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Y_Quality 로 새로운 열 어떻게 생성할지
- LINE이랑 합쳐서 뭘 채울지
- PRODUCT_CODE랑 합쳐서 뭘 채울지

비식별화된 X변수들 어떻게 할지 고민(나중 맨 마지막에)
- PCA를 쓰기도 하던데 보류
- scaling
- 이상치 확인하고 제거

## Data Pre-processing

In [ ]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [ ]:
# 비식별 X features의 std값이 0인 열 삭제
X_feat = train_x.iloc[:, 2:].describe()
drop_col = X_feat.columns[X_feat.loc['std'] == 0]
train_x = train_x.drop(drop_col, axis=1)
test_x = test_x.drop(drop_col, axis=1)

In [ ]:
# 281 개의 열이 삭제됨
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_4,X_5,X_7,X_8,X_9,X_10,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,0.0
1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,0.0
2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,0.0
3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,0.0
4,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,0.0


## 파생변수 생성

In [ ]:
X_feat = train_x.iloc[:, 2:]
# print(X_feat.mean(axis=1))
train_x['X_feat_mean'] = X_feat.mean(axis=1)
train_x['X_feat_std'] = X_feat.std(axis=1)
train_x['X_feat_median'] = X_feat.median(axis=1)

X_feat = test_x.iloc[:, 2:]
test_x['X_feat_mean'] = X_feat.mean(axis=1)
test_x['X_feat_std'] = X_feat.std(axis=1)
test_x['X_feat_median'] = X_feat.median(axis=1)

## Modeling

In [ ]:
!pip install catboost

In [ ]:
from xgboost import XGBClassifier

xgbc = XGBClassifier(random_state=37)

In [ ]:
xgbc.fit(train_x, train_y)

XGBClassifier(objective='multi:softprob', random_state=37)

In [ ]:
pred = xgbc.predict(test_x)

# Submit

In [ ]:
# 제출 파일을 불러옵니다.
submission = pd.read_csv("./sample_submission.csv")

In [ ]:
submission['Y_Class'] = pred
submission

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,1
4,TEST_004,1
...,...,...
305,TEST_305,1
306,TEST_306,1
307,TEST_307,1
308,TEST_308,1


In [ ]:
submission.to_csv('./ProductQuality(6).csv', index=False)